In [1]:
#code created by Carl van Beek
#last updated on 16-10-2023

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [2]:
#load data
#Data is in long format with columns for EEG spectral power values, PIW, Order, Subject, Trial number, and Condition.

Alpha_norm_table = pd.read_csv("C:/Users/.../Table_alpha_baselinecorrectedvalues.csv")
Beta_norm_table = pd.read_csv("C:/Users/.../Table_beta_baselinecorrectedvalues.csv")
Theta_norm_table = pd.read_csv("C:/Users/.../Table_theta_baselinecorrectedvalues.csv")
Engagement_norm_table = pd.read_csv("C:/Users/.../Table_engagement_baselinecorrectedvalues.csv")

Alpha_raw_table = pd.read_csv("C:/Users/.../Table_alpha_rawvalues.csv")
Beta_raw_table = pd.read_csv("C:/Users/.../Table_beta_rawvalues.csv")
Theta_raw_table = pd.read_csv("C:/Users/.../Table_theta_rawvalues.csv")
Engagement_raw_table = pd.read_csv("C:/Users/.../Table_engagement_rawvalues.csv")

PIW_Table = pd.read_csv("C:/Users/.../Table_PIW.csv")

In [5]:
#create tables containing only features

new_Alpha_norm_table = Alpha_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Beta_norm_table = Beta_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Theta_norm_table = Theta_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Engagement_norm_table = Engagement_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])

new_Alpha_raw_table = Alpha_raw_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Beta_raw_table = Beta_raw_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Theta_raw_table = Theta_raw_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Engagement_raw_table = Engagement_raw_table.drop(columns = ['Order','Subject','Trial', 'Condition'])

new_PIW_Table = PIW_Table["OneD_PIW_longitudinal"]


#concatenate all features into a single feature table, making a distinction between a table with and without stick data
full_EEG_norm_table = pd.concat([new_Alpha_norm_table,new_Beta_norm_table,new_Theta_norm_table,new_Engagement_norm_table], axis=1)
full_PIW_EEG_norm_table = pd.concat([full_EEG_norm_table, new_PIW_Table], axis = 1)


full_EEG_raw_table = pd.concat([new_Alpha_raw_table,new_Beta_raw_table,new_Theta_raw_table,new_Engagement_raw_table], axis=1)
#full_raw_table = pd.concat([full_EEG_raw_table,new_duty_aggr_table], axis = 1 )

#get features
y = Alpha_norm_table['Condition'] # prediction labels, they are the same in all tables, alpha_norm is just randomly used


function_test_table = pd.concat([full_PIW_EEG_norm_table,y], axis = 1)

print(function_test_table)

    Alpha_FP1_norm  Alpha_FP2_norm  Alpha_AF3_norm  Alpha_AF4_norm  \
0        -0.001327        0.010385       -0.171520       -0.119126   
1        -0.037265       -0.043328       -0.252725       -0.242479   
2        -0.057347       -0.063372       -0.189426       -0.205583   
3         0.029274        0.005957       -0.012776       -0.026032   
4        -0.029137       -0.043072       -0.018884       -0.070772   
..             ...             ...             ...             ...   
67        0.411114        0.729160        0.326688        1.309336   
68        0.674143        1.021150        0.174724        1.514810   
69        0.126088        0.174602        0.224063        1.186826   
70        0.110539        0.168365        0.331968        2.852336   
71        0.138000        0.198183        0.405219        2.631847   

    Alpha_F7_norm  Alpha_F3_norm  Alpha_Fz_norm  Alpha_F4_norm  Alpha_F8_norm  \
0       -0.090453      -0.009004      -0.103003      -0.047938      -0.116356 

In [8]:
#Dataframe should have columns: 'Condition' and Alpha, Beta, Theta and Engagement values
#for all channels. In the case of 12 x 6 trials and 32 channels, this would be a 72 rows x 129 columns dataframe
#Order, Subject and Trial are not used in this function, but could be used in other applications
def workload_predictor(df):
    X_EEG = df.drop(columns = ['Condition'])
    #Make sure there is a column called 'Condition'. Could be replaced by an integer if there is a standard for that.
    y = df['Condition']
    #first create the RFE algorithm
    SVM = SVC(kernel = 'linear',C = 1,gamma = 0.1) #Choose a classifier to perform RFE
    SVM_RFE = RFE(SVM,n_features_to_select = 10) #Perform RFE
    SVM_RFE.fit(X_EEG,y) 

    #View which features got selected
    selected_EEG = X_EEG.columns[SVM_RFE.support_]
    print("Selected features: ", selected_EEG)


    #Transform the original X-data to only retain the selected features
    X_RFE_EEG = SVM_RFE.transform(X_EEG)

    #global value to keep track of the accuracy of each iteration
    all_accuracies_EEG = []
    all_f1_EEG = []
    all_precision_EEG = []
    all_recall_EEG = []

    #perform the training and testing 10 times, to rule out chance due to the small dataset
    for i in range(1,11):
        #split into 70-30 training and testing for RFE
        X_RFE_train_EEG, X_RFE_test_EEG, y_train, y_test = train_test_split(X_RFE_EEG, y, test_size = 0.3)

        #gridsearch for all models of the first level in the stacking classifier
        rf_params = {'n_estimators': [10, 25, 50, 100,200], 'max_depth': [None, 5, 10, 15]}
        RF_EEG = RandomForestClassifier() #Check literature
        rf_grid_EEG = GridSearchCV(RF_EEG, rf_params, cv=5, n_jobs=-1)
        rf_grid_EEG.fit(X_RFE_train_EEG,y_train)

        svm_params = {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10, 100]}
        svm_EEG = SVC()
        svm_grid_EEG = GridSearchCV(svm_EEG, svm_params, cv=5, n_jobs=-1)
        svm_grid_EEG.fit(X_RFE_train_EEG,y_train)

        lr_params = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['none','l2']}
        lr_EEG = LogisticRegression()
        lr_grid_EEG = GridSearchCV(lr_EEG, lr_params, cv=5, n_jobs=-1)
        lr_grid_EEG.fit(X_RFE_train_EEG,y_train)

        #variable to input as the first level in the stacking classifier
        estimators_EEG = [('rf', rf_grid_EEG),('svc', svm_grid_EEG),('lr', lr_grid_EEG)]

        #gridsearch for the second level of the stacking model
        second_level_params_EEG = {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10, 100]}
        second_level_model_EEG = SVC()
        second_level_grid_EEG = GridSearchCV(second_level_model_EEG, second_level_params_EEG, cv=5, n_jobs=-1)
        second_level_grid_EEG.fit(X_RFE_train_EEG,y_train)

        #Create the stacking classifier
        sclf_EEG = StackingClassifier(estimators=estimators_EEG, final_estimator=second_level_grid_EEG)
        sclf_EEG.fit(X_RFE_train_EEG, y_train)

        prediction = sclf_EEG.predict(X_RFE_test_EEG)
        #get model perfromance
        accuracy_EEG = accuracy_score(y_test, prediction)
        f1_EEG = f1_score(y_test, prediction)
        precision_EEG = precision_score(y_test, prediction)
        recall_EEG = recall_score(y_test, prediction)
        #add the performance metrics to their global list to obtain averages later
        all_accuracies_EEG.append(accuracy_EEG)
        all_f1_EEG.append(f1_EEG)
        all_precision_EEG.append(precision_EEG)
        all_recall_EEG.append(recall_EEG)
        #print out the metrics for the current iteration
        print("Accuracy test number ", i, ": ", accuracy_EEG)
        print("F1 score test number ", i, ": ", f1_EEG)
        print("Precision test number ", i, ": ", precision_EEG)
        print("Recall test number ", i, ": ", recall_EEG)
        #obtain the chosen hyperparameters
        print("Chosen parameters for the random forest model in the first level of the stacking classifier for iteration number ",i,": ",rf_grid_EEG.best_params_)
        print("Chosen parameters for the SVM model in the first level of the stacking classifier for iteration number ",i,": ",svm_grid_EEG.best_params_)
        print("Chosen parameters for the Logistic regression model in the first level of the stacking classifier for iteration number ",i,": ",lr_grid_EEG.best_params_)
        print("Chosen parameters for the SVM model in the second level of the stacking classifier for iteration number ",i,": ",second_level_grid_EEG.best_params_)

    #print out the average model evaluation scores of all iterations
    print("Average accuracy score", np.mean(all_accuracies_EEG))
    print("Average f1 score", np.mean(all_f1_EEG))
    print("Average precision", np.mean(all_precision_EEG))
    print("Average recall", np.mean(all_recall_EEG))

    

In [9]:
workload_predictor(function_test_table)

Selected features:  Index(['Alpha_C4_norm', 'Alpha_P8_norm', 'Alpha_PO8_norm', 'Beta_AF3_norm',
       'Beta_AF4_norm', 'Beta_F8_norm', 'Beta_P7_norm', 'Beta_P4_norm',
       'Beta_PO4_norm', 'Beta_PO8_norm', 'Beta_Oz_norm', 'Theta_FP1_norm',
       'Theta_Cz_norm', 'Theta_CP6_norm', 'Theta_Pz_norm', 'Theta_Oz_norm',
       'Engagement_AF3_norm', 'Engagement_FC2_norm', 'Engagement_T8_norm',
       'Engagement_P4_norm'],
      dtype='object')


KeyboardInterrupt: 